# Cooking Assistant

## openai
The library that will make it easier for us to connect directly to the openAI API.

## os and dotenv
The libraries that allow me to upload this directly to github without exposing my API credentials.

In [1]:
import csv
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

## Connecting to the openAI API

In [3]:
openai.api_key = os.getenv("OPEN_API_KEY")

## generate_recipe
This function formats the prompt that is sent to GPT3 (text-davinci-003). It takes variables that will eventually be available inputs on a webpage.

In [4]:
def generate_recipe(cooking_skill, place_city, preparation_time, time_of_day):
    prompt = f"Please provide a detailed {time_of_day} cooking recipe with ingredients and step-by-step directions, suitable for someone with {cooking_skill}, originating from {place_city}, and taking {preparation_time} time to prepare and cook. Please also keep it under 10 ingredients. \nPlease provide this in JSON format with the following variables:\nname, description, ingredients, directions. Do not return 'difficulty, origin, time'"

    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=1000,
        n=1,
        temperature=0,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0
    )
    # return response.choices[0].text.strip()
    return response

In [5]:
cooking_skill = "Advanced"
place_city = "Detroit"
preparation_time = "Short"
time_of_day = "Lunch"

## recipe_json
Here we will be executing the function by passing our 3 variables into the prompt, which is then sent to openAPI

In [6]:
recipe = generate_recipe(cooking_skill, place_city, preparation_time, time_of_day)

## JSON Structure
This JSON response will be used to fill in our webpage in an digestible format.

In [7]:
print(recipe)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n{\n    \"name\": \"Detroit-Style Coney Dogs\",\n    \"description\": \"A classic Detroit-style lunch that can be prepared and cooked in a short amount of time.\",\n    \"ingredients\": [\n        \"1 lb. ground beef\",\n        \"1/2 cup diced onion\",\n        \"1/2 cup diced green bell pepper\",\n        \"1/4 cup ketchup\",\n        \"1/4 cup yellow mustard\",\n        \"1/4 cup chili sauce\",\n        \"1/4 cup Worcestershire sauce\",\n        \"1/2 teaspoon garlic powder\",\n        \"1/2 teaspoon onion powder\",\n        \"8 hot dog buns\"\n    ],\n    \"directions\": [\n        \"In a large skillet over medium-high heat, cook the ground beef, onion, and bell pepper until the beef is browned and the vegetables are tender, about 8 minutes.\",\n        \"Add the ketchup, mustard, chili sauce, Worcestershire sauce, garlic powder, and onion powder. Stir to combine and co

In [8]:
strip_recipe = recipe.choices[0].text.strip()

In [9]:
import json

In [10]:
recipe_json = json.loads(strip_recipe)

In [11]:
# filename = 'pastresponse.csv'

In [12]:
# with open(filename, 'a', newline='') as past_responses:
#     fieldnames = ['name', 'description', 'ingredients', 'directions']
#     writer = csv.DictWriter(past_responses, fieldnames=fieldnames)
#     writer.writerow({
#         'name': recipe_json['name'],
#         'description': recipe_json['description'],
#         'ingredients': ", ".join(recipe_json['ingredients']),
#         'directions': ", ".join(recipe_json['directions'])
#     })

In [13]:
from supabase import create_client, Client

url = os.getenv("SUPABASE_URL")
supakey = os.getenv("SUPABASE_API_KEY")
supabase = create_client(url, supakey)

In [24]:
suparesponse = supabase.table('main').select("name").execute()

In [25]:
suparesponse

APIResponse(data=[{'name': 'Waco-Style BBQ Chicken Sandwich'}, {'name': 'Czech Potato Salad'}, {'name': 'Czech Potato Salad'}, {'name': 'Miami-Style Grilled Fish Tacos'}, {'name': 'Quebec-Style French Toast'}, {'name': 'Detroit-Style Coney Dogs'}], count=None)

In [16]:
data = supabase.auth.sign_in_with_password({"email": os.getenv("SUPABASE_EMAIL"), "password": os.getenv("SUPABASE_LOGIN")})

In [17]:
table = supabase.table('main')

In [18]:
insert_query = table.insert({
    'name': recipe_json['name'],
    'description': recipe_json['description'],
    'ingredients': ", ".join(recipe_json['ingredients']),
    'directions': ", ".join(recipe_json['directions']),
    'cooking_skill': cooking_skill,
    'place_city': place_city,
    'preparation_time': preparation_time,
    'time_of_day': time_of_day
})

In [19]:
result = insert_query.execute()

In [20]:
recipe.usage['total_tokens']

336